In [ ]:
import requests
from io import BytesIO
from PyPDF2 import PdfReader
import openai
import re
import json
import os
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
arxiv_url = "https://arxiv.org/pdf/2110.01111.pdf"

def get_pdf_object(url):
    response = requests.get(arxiv_url)
    reader = PdfReader(BytesIO(response.content))
    return reader

def get_pdf_text(url):
    response = requests.get(arxiv_url)
    reader = PdfReader(BytesIO(response.content))
    corpus = ''
    for i in reader.pages:
        corpus += i.extract_text()
    # remove newlines
    # corpus = corpus.replace("\x03", "").replace("\n", "")
    return corpus

corpus = get_pdf_text(arxiv_url)
pdf = get_pdf_object(arxiv_url)
corpus

In [ ]:
def parse_paper(pdf):
  print("Parsing paper")
  number_of_pages = len(pdf.pages)
  print(f"Total number of pages: {number_of_pages}")
  paper_text = []
  for i in range(number_of_pages):
    page = pdf.pages[i]
    page_text = []

    def visitor_body(text, cm, tm, fontDict, fontSize):
      x = tm[4]
      y = tm[5]
      # ignore header/footer
      if (y > 50 and y < 720) and (len(text.strip()) > 1):
        page_text.append({
          'fontsize': fontSize,
          'text': text.strip().replace('\x03', ''),
          'x': x,
          'y': y
        })

    _ = page.extract_text(visitor_text=visitor_body)

    blob_font_size = None
    blob_text = ''
    processed_text = []

    for t in page_text:
      if t['fontsize'] == blob_font_size:
        blob_text += f" {t['text']}"
      else:
        if blob_font_size is not None and len(blob_text) > 1:
          processed_text.append({
            'fontsize': blob_font_size,
            'text': blob_text,
            'page': i
          })
        blob_font_size = t['fontsize']
        blob_text = t['text']
    paper_text += processed_text
  return paper_text

In [ ]:
paper = parse_paper(pdf)

In [ ]:
def paper_df(pdf):
    filtered_pdf= []
    for row in pdf:
        if len(row['text']) < 30:
            continue
        filtered_pdf.append(row)
    df = pd.DataFrame(filtered_pdf)
    df['length'] = df['text'].apply(lambda x: len(x))
    print(df.shape)
    return df



In [ ]:
df = paper_df(paper)
df.head()

In [ ]:
def calculate_embeddings(df):
    embedding_model = "text-embedding-ada-002"
    embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
    df["embeddings"] = embeddings
    return df

calculate_embeddings(df)

In [ ]:
def search_reviews(df, query, n=3, pprint=True):
    query_embedding = get_embedding(
        query,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))

    results = (
        df.sort_values("similarity", ascending=False, ignore_index=True)
        
    )
    return results.head(n)

In [ ]:
results = search_reviews(df, "How many swaps can the algorithm make?", n=3)
results

In [ ]:
# number_of_pages = len(pdf.pages)
# paper_text = []
# for i in range(number_of_pages):
#     page = pdf.pages[i]
#     page_text = []

#     def visitor_body(text, cm, tm, fontDict, fontSize):
#         x = tm[4]
#         y = tm[5]
#         # ignore header/footer
#         if (y > 50 and y < 720) and (len(text.strip()) > 1):
#             page_text.append({
#             'fontsize': fontSize,
#             'text': text.strip().replace('\x03', ''),
#             'x': x,
#             'y': y
#             })

#     _ = page.extract_text(visitor_text=visitor_body)

#     blob_font_size = None
#     blob_text = ''
#     processed_text = []

#     for t in page_text:
#         if t['fontsize'] == blob_font_size:
#             blob_text += f" {t['text']}"
#         else:
#             if blob_font_size is not None and len(blob_text) > 1:
#                 processed_text.append({
#                     'fontsize': blob_font_size,
#                     'text': blob_text,
#                     'page': i
#                 })
#             blob_font_size = t['fontsize']
#             blob_text = t['text']
#         paper_text += processed_text
# print("Done parsing paper")

# filtered_pdf= []
# for row in paper_text:
#     if len(row['text']) < 30:
#         continue
#     filtered_pdf.append(row)
# df = pd.DataFrame(filtered_pdf)
# # print(df.head())
# df['length'] = df['text'].apply(lambda x: len(x))
# # print(df.shape)
# print('Done creating dataframe')

# openai.api_key = os.getenv('OPENAI_API_KEY')
# embedding_model = "text-embedding-ada-002"
# print('Calculating embeddings')
# print(df.text)
# embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
# df["embeddings"] = embeddings
# print('Done calculating embeddings')
# user_input = "What is GPT-3"
# query_embedding = get_embedding(
#     user_input,
#     engine="text-embedding-ada-002"
# )
# df["similarity"] = df.head().embeddings.apply(lambda x: cosine_similarity(x, query_embedding))

# results = (
#     df.sort_values("similarity", ascending=False)
    
# )

# prompt = """You are a large language model whose expertise is reading and summarizing scientific papers. 
#     You are given a query and a series of text embeddings from a paper in order of their 
#     cosine similarity to the query. You must take the given embeddings and return a very detailed summary of the paper 
#     that answers the query.
    
#     Given the query"""+ user_input + """and the following embeddings: 
    
#     1.""" + results.iloc[0] + """
#     2.""" + results.iloc[1] + """
#     3.""" + results.iloc[3] + """

#     Return a detailed answer based on the paper that answers the query."""
# print('Generating response from GPT-3')

# openai.api_key = os.getenv('OPENAI_API_KEY')
# r = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0.4, max_tokens=2000)
# response = r.choices[0]['text']



In [ ]:
def section_headers(corpus):
    section_headers = re.findall(r'\n\d+\s+(.*?)\n', corpus)
    filtered_headers = list(filter(lambda x: x[0].isupper() and not x.isnumeric() and len(x.split())<=4, section_headers))
    return filtered_headers

headers = section_headers(corpus)
headers

In [ ]:
# section_texts = re.findall(r'\n\d+\s+(.*?)\n(.*?)\n', corpus, re.DOTALL)
# section_texts

In [ ]:
# regex to get paragraphs in between the section headers
paragraphs = re.findall(r'\n\d+\s+(.*?)\n(.*?)\n\d+\s+(.*?)\n', corpus, re.DOTALL)
paragraphs

In [ ]:
# total number of characters in paragraphs
sum([len(i[1]) for i in paragraphs])

In [ ]:
prompt = """You are a large language model whose expertise is reading and summarizing scientific papers. 
            You are given a query and a series of text embeddings from a paper in order of their 
            cosine similarity to the query. You must take the given embeddings and return a very detailed summary of the paper 
            that answers the query.
            
            Given the query"""+ query + """and the following embeddings: 
            
            1.""" + embedding1 + """
            2.""" + embedding2 + """
            3.""" + embedding3 + """

            Return a detailed answer based on the paper that answers the query."""

In [ ]:
def gpt(prompt):
    openai.api_key = os.getenv('OPENAI_API_KEY')
    r = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0.4, max_tokens=2000)
    response = r.choices[0]['text']

    return response

In [ ]:
summary = gpt(prompt)


In [ ]:
def summarize(corpus):
    prompt = """Here is the abstract from a recent machine learning paper:'"""+corpus+"""'
            Summarize the above content in detail in the style of an excited professor talking about this cool paper he just read.
            Summarized content:"""
    summary = gpt(prompt)
    summary = summary.replace('\n', '')
    return summary

In [ ]:
# summary = "I just read an amazing paper on a new machine learning algorithm called Dreamer V three. It's a general and scalable algorithm based on world models, and it outperforms previous approaches across a wide variety of domains. It can handle continuous and discrete actions, visual and low-dimensional inputs, two D and three D worlds, different data budgets, reward frequencies, and reward scales. Plus, it has favorable scaling properties, with larger models resulting in higher data-efficiency and performance. And here's the best part: it's the first algorithm to collect diamonds in Minecraft from scratch without human data or curricula - a long-standing challenge in artificial intelligence. This algorithm makes reinforcement learning broadly applicable and allows scaling to hard decision-making problems. It's really exciting and I can't wait to see what else it can do!"

In [ ]:
def chop(text, chunk_size):
    chunks = []
    current_chunk = ""
    for word in text.split():
        if len(current_chunk) + len(word) + 1 > chunk_size:
            chunks.append(current_chunk)
            current_chunk = ""
        current_chunk += word + " "
    chunks.append(current_chunk)
    return chunks

chunks = chop(summary, 2000)

In [ ]:
len(chunks[-1])

In [ ]:
targets = ["https://mukuls-public-playground.s3.us-east-2.amazonaws.com/jre.mp3", "https://mukuls-public-playground.s3.us-east-2.amazonaws.com/jre2.mp3", "https://mukuls-public-playground.s3.us-east-2.amazonaws.com/jre3.mp3"]

In [ ]:
def cast(text, targets):
  headers = {
    "Content-Type": "application/json",
    "Connection": "keep-alive",
    "Keep-Alive": "timeout=1000, max=100",
  }

  data = json.dumps({
    "api_key": "ff8f47e8-3643-44bc-b9d3-de51142b95fd",
    "text": text,
    "voices": "",
    "target_file": targets,
  })

  response = requests.post(
    'https://vatsalaggarwal--tts-app.modal.run',
    headers=headers,
    data=data
  )

  # Returned audio is 24kHz 32-bit PCM WAV file
  # with open('jre3.wav', 'wb') as f:
  #     f.write(response.content)
  return response.content

In [ ]:
# # Concatenate the audio files

# from pydub import AudioSegment

# def concatenate_audio_files(file_names, output_file_name):
#     # Initialize an empty audio segment
#     output = AudioSegment.empty()
#     # Iterate over the file names
#     for file_name in file_names:
#         # Load the audio file
#         audio = AudioSegment.from_wav(file_name)
#         # Concatenate the audio file to the output
#         output += audio
#     # Save the output to a new file in the folder 'outputs' that is located in the same directory as this notebook
#     output.export('outputs/' + output_file_name, format='wav') 




# # Get the file names of the audio files
# file_names = ['jre.wav', 'jre2.wav', 'jre3.wav']

# concatenate_audio_files(file_names, 'jre_concat.wav')